### 중급반 180114
1.비정형데이터- 텍스트
2.대용량데이터- 100MB 놋북수준
# 
[실습코드](https://goo.gl/A7VE3a)
https://goo.gl/A7VE3a

다음주0121은 강사님 생일이라 PASS.
텍스트는 워낙 할 일이 많아서 협업이 중요!
Kor에서 못 하는 건 안 할 것.
로튼 토마토- 공신력 높은 영화리뷰 사이트.

[영화감정분석](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data)
평점말고, 텍스트만으로 영화평이 긍/부정인지 알아내기
# 
socher- 예전1위. yoon kim- 현재1위. 조만간 방한예정

문장을 단어로쪼갬. 특정 단어있나없나& 빈도. 를 머신러닝 모델에 맡김
# 
phrase: 나는 딥러닝이 좋습니다
# 
sentiment/ 나는/ 딥러닝이/ ~/ 좋습니다
# 
4/ 1/ 1/ 0/ 1 -> '좋습니다' 들어가있으면 sentiment가 높구나! 를 추정

In [1]:
import pandas as pd

In [3]:
%cd data
%ls
%cd ..

[WinError 2] 지정된 파일을 찾을 수 없습니다: 'data'
D:\github\dataAnal_py_basic\dss_sentimentMovie
 D 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 504F-F621

 D:\github\dataAnal_py_basic\dss_sentimentMovie 디렉터리

2018-04-27  오후 01:04    <DIR>          .
2018-04-27  오후 01:04    <DIR>          ..
2018-02-28  오후 11:27    <DIR>          .ipynb_checkpoints
2018-02-28  오후 11:27           596,647 baseline-script.csv
2018-02-28  오후 11:27           596,647 baseline-script_my1_CVec.csv
2018-02-28  오후 11:27            43,120 movie_2ndWeek_nltk.ipynb
2018-02-28  오후 11:27           108,863 movie_2ndWeek_nltk_T_base.ipynb
2018-02-28  오후 11:27            76,423 movie_shinOhter_nltk_.ipynb
2018-02-28  오후 11:27            54,410 movie-xgboost_T.ipynb
2018-02-28  오후 11:27            19,568 NLTK_T.ipynb
2018-02-28  오후 11:27            16,623 nltk_t_down.ipynb
2018-02-28  오후 11:27           180,229 result.csv
2018-02-28  오후 11:27            61,994 SentimentAnalysis_movie_myCountVectorize.ipynb
2018-02-28  오후 11:27            51,928

In [4]:
#tap으로 구분. seperate. index_col 없으면 판다가 0,1,2...를 만듦
train= pd.read_csv("data/train.tsv", sep="\t", index_col="PhraseId")
test= pd.read_csv("data/test.tsv", sep="\t", index_col="PhraseId")

데이터가 많이 필요하니 영화리뷰 하나를 여러개로 쪼갬. 아마존_메카니컬 텀프_데이터 생성 노가다

In [19]:
#sentiment 평점. 4점 높
print(train.shape)
train.head(10)

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2
6,1,of escapades demonstrating the adage that what...,2
7,1,of,2
8,1,escapades demonstrating the adage that what is...,2
9,1,escapades,2


In [18]:
print(test.shape)
test.head(10)

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine
156066,8545,intermittently pleasing but
156067,8545,intermittently pleasing
156068,8545,intermittently
156069,8545,pleasing


### Preprocessing
#### Vectorize Phrases
자연어처리 기능이 대단히 많음

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# max_feature=1000: 상위 천개만 남기겠다
vectorizer= CountVectorizer(max_features=1000)
vectorizer
# stop_word: 빈번하게 발생하는 단어. None_디폴트 체크리스트. 따로 넣어줄수도 있음- 한글처리 할때

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [37]:
#fit / transform
# fit: 빈도수를 기준으로 뭐가 상위인지 결정
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [38]:
X_train= vectorizer.transform(train["Phrase"])
X_train

<156060x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 653246 stored elements in Compressed Sparse Row format>

In [39]:
# get_feature_names: 
vocabulary= vectorizer.get_feature_names()
# 보기좋게 만들기. 이 단어는 요 단어가 몇 번 나왔는지
# 0:1000 - 1천개까지만 보자
pd.DataFrame(X_train[0:1000].toarray(), columns= vocabulary).head()

,10,20,2002,90,ability,able,about,above,across,act,...,written,wrong,ya,year,years,yet,york,you,young,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X_test= vectorizer.transform(test["Phrase"])
X_test

<66292x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 254492 stored elements in Compressed Sparse Row format>

In [41]:
y_train= train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

### Train
경험상 여기선 tree보단 regression model이 좋더라. tree는 느릴것

#### 각 모델의 장단점?
regression: 안 중요하다 싶으면 0점 줌
{0:.6f}, {1:.6f}, .format(w, error)

In [42]:
from sklearn.linear_model import SGDClassifier

In [45]:
# SGDClassifier: 엄청 많이 쓰는거. 결과가 랜덤하게 나옴. random_state로 고정_seed number
model= SGDClassifier(random_state= 37)
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

### Scoring
Cross validation: data 조각내서 4개로 학습하고 1개로 예측. 을 5번 반복

In [ ]:
# from sklearn.model_selection import cross_val_score

In [50]:
# # error는 무시- 맨밑에 결과 뜸. cv: 조각수
# score= cross_val_score(model, X_train, y_train, cv=5).mean()
# print("Score: {0:.6f}".format(score))

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

Score: 0.557061


In [60]:
from sklearn.model_selection import cross_val_predict

In [62]:
y_predict= cross_val_predict(model, X_train, y_train, cv=5)
print(y_predict.shape)
y_predict

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

(156060,)


array([1, 3, 2, ..., 2, 2, 2], dtype=int64)

Score 0.55 : 55% - 대부분 2_중립 이라. 이게 최소값. 평점 0~4점을 맞출 확률
# 
분석: 선택과 집중. 맞춘건 더 맞출필요없고, 틀린걸 맞추자

1. 맞춘걸 아래로 놓고, 틀린걸 위로 놓고
2. 틀린것중에서 많이 틀린것(1점인데 4점으로 예측한거)은 더 위로 정렬

### Predict

In [51]:
model.fit(X_train, y_train)

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [54]:
predictions= model.predict(X_test)

In [55]:
print(predictions.shape)
predictions

(66292,)


array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

### Submit

In [57]:
submission= pd.read_csv("data/sampleSubmission.csv", index_col="PhraseId")

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [58]:
submission["Sentiment"]= predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


#### 내보내기

In [59]:
submission.to_csv("baseline-script.csv")